In [1]:
import pandas as pd
import numpy as np
import zipfile

## Import Data

In [2]:
# ! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Parcel.zip
# ! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Real%20Property%20Sales.zip
# ! wget -P ../../data/raw https://aqua.kingcounty.gov/extranet/assessor/Residential%20Building.zip

In [3]:
# with zipfile.ZipFile('../../data/raw/Parcel.zip', 'r') as zip_ref:
#     zip_ref.extractall('../../data/raw')
# with zipfile.ZipFile('../../data/raw/Real Property Sales.zip', 'r') as zip_ref:
#     zip_ref.extractall('../../data/raw')
# with zipfile.ZipFile('../../data/raw/Residential Building.zip', 'r') as zip_ref:
#     zip_ref.extractall('../../data/raw')

In [ ]:
parcel = pd.read_csv("../../data/raw/EXTR_Parcel.csv", encoding='latin-1')
real_property_sales = pd.read_csv("../../data/raw/EXTR_RPSale.csv", encoding='latin-1', low_memory=False)
residential_building = pd.read_csv("../../data/raw/EXTR_ResBldg.csv", encoding='latin-1', low_memory=False)

## Clean Data and Filter For Needed Info

### Property Sales

In [ ]:
# Real Property Sales
# Identifyers are Major and Minor. Relevant data is SalePrice
# Clean real_property_sales to have proper values in Minor and Major and change type to int to match other data sets
real_property_sales = real_property_sales[['Major', 'Minor', 'SalePrice', 'DocumentDate']]
real_property_sales = real_property_sales[(real_property_sales['Major'].str.isdecimal()==True) &
                                          (real_property_sales['Minor'].str.isdecimal()==True)]
real_property_sales['Major'] = real_property_sales['Major'].astype('int64')
real_property_sales['Minor'] = real_property_sales['Minor'].astype('int64')
real_property_sales = real_property_sales[real_property_sales['SalePrice']>0]

In [ ]:
real_property_sales.head()

### Parcel

In [ ]:
# parcel.info()

In [ ]:
# Parcel Filter for Necessary Columns
parcel = parcel[['Major', 'Minor', 'PropType', 'SqFtLot', 'WfntLocation',
                 'TrafficNoise', 'AirportNoise', 'PowerLines', 'OtherNuisances',
                 'MtRainier', 'Olympics', 'Cascades', 'PugetSound', 'LakeWashington',
                 'LakeSammamish', 'SmallLakeRiverCreek',]]
parcel.info()

In [ ]:
parcel = parcel[(parcel['PropType']=='R') | (parcel['PropType']=='K')]

### Residential Buiding

In [ ]:
# residential_building.info()

In [ ]:
# Residential Buiding Filter For Necessary Columns
residential_building = residential_building[['Major', 'Minor', 'ZipCode', 'NbrLivingUnits', 'SqFtTotLiving', 'SqFtOpenPorch', 'SqFtEnclosedPorch']]

In [ ]:
# Create a column Has_Porch
residential_building['Has_Porch']= residential_building['SqFtOpenPorch'] + residential_building['SqFtEnclosedPorch']
residential_building['Has_Porch'] = [1  if p>0 else 0 for p in residential_building['Has_Porch']]
residential_building['Has_OpenPorch'] = [1  if p>0 else 0 for p in residential_building['SqFtOpenPorch']]
residential_building['Has_EnclosedPorch'] = [1  if p>0 else 0 for p in residential_building['SqFtEnclosedPorch']]

In [ ]:
residential_building.info()

### Merge Property Sales, parcel, and residential building data

In [ ]:
# Join all the tables on Major, Minor
# Have duplicate buildings sold at different dates. Need to only keep latest dates
merged_tables = (parcel.merge(real_property_sales, on = ['Major', 'Minor'])).merge(residential_building, on = ['Major', 'Minor']).drop_duplicates()
# Change dates to date time format
merged_tables[['Major', 'SqFtLot']].head(20)
merged_tables['DocumentDate'] = pd.to_datetime(merged_tables['DocumentDate'])

In [ ]:
merged_tables.shape

In [ ]:
#Keeping only the most recent Sale_Price

merged_tables = merged_tables.sort_values(by=['Major', 'DocumentDate'])
merged_tables.drop_duplicates(subset=['Major', 'Minor'], keep='last', inplace=True)

In [ ]:
# Filtering for only 2019 data
start_date = "2019-1-1"
end_date = "2019-12-31"
merged_tables = merged_tables.loc[(merged_tables["DocumentDate"] >= start_date) & (merged_tables["DocumentDate"] <= end_date)]

merged_tables.shape

In [ ]:
# change dataframe name to df to make life easier
df = merged_tables #.sort_values(by = 'DocumentDate', ascending=False)

#Create Nuisances Column to replace other nuisance columns
df['Nuisances'] = 'No Nuisances'
df.loc[(df['TrafficNoise']>0), 'Nuisances'] = 'Traffic Noise'
df.loc[(df['AirportNoise']>0), 'Nuisances'] = 'Airport Noise'
df.loc[(df['PowerLines']=='Y'), 'Nuisances'] = 'Power Lines'
df.loc[(df['OtherNuisances']=='Y'), 'Nuisances'] = 'Other Nuisances'
df = df.drop(['PropType', 'AirportNoise', 'SqFtOpenPorch', 'SqFtEnclosedPorch'], axis='columns')
df['PowerLines'] = df['PowerLines'].map({'Y' : 1, 'N' : 0})
df['OtherNuisances'] = df['OtherNuisances'].map({'Y' : 1, 'N' : 0})

In [ ]:
# WFNTLOCATION
# Waterfront location (Source: King County Assessments)
# Value	Definition
# 0	Unknown
# 1	Duwamish
# 2	Elliott Bay
# 3	Puget Sound
# 4	Lake Union
# 5	Ship Canal
# 6	Lake Washington
# 7	Lake Sammamish
# 8	Other lake
# 9	River or slough

df['WaterFrontLocation'] = df.WfntLocation.map({0 : 'None', 1 : 'Duwamish', 2 : 'Elliott Bay', 3 : 'Puget Sound', 4 : 'Lake Union',
                     5 : 'Ship Canal',  6 : 'Lake Washington', 7 : 'Lake Sammamish', 8 : 'Other lake', 9 : 'River or slough'})
df['Is_WaterFrontLocation'] = [1 if i>0 else 0 for i in df.WfntLocation]
df=df.drop('WfntLocation', axis='columns')

In [ ]:
df['Has_Nuisance'] = [0 if n == 'No Nuisances' else 1 for n in df['Nuisances']]

# df[['Major', 'Minor', 'SqFtLot']].head()

In [ ]:
# reorder Columns
df = df[['Major', 'Minor', 'ZipCode', 'DocumentDate', 'SalePrice', 'SqFtLot', 'SqFtTotLiving',
         'Nuisances', 'Has_Nuisance', 'TrafficNoise', 'PowerLines', 'OtherNuisances',
         'Is_WaterFrontLocation', 'WaterFrontLocation', 'Has_Porch', 'Has_OpenPorch',
         'Has_EnclosedPorch', 'NbrLivingUnits', 'MtRainier', 'Olympics', 'Cascades',
         'PugetSound', 'LakeWashington', 'LakeSammamish', 'SmallLakeRiverCreek']]

In [ ]:
df.rename(columns={'MtRainier': 'View_MtRainier', 'Olympics': 'View_Olympics', 'Cascades': 'View_Cascades',
                   'PugetSound': 'View_PugetSound', 'LakeWashington': 'View_LakeWashington',
                   'LakeSammamish': 'View_LakeSammamish','SmallLakeRiverCreek': 'View_SmallLakeRiverCreek',}, inplace = True)

In [ ]:
# Add in and clean zipcode column

df['ZipCode'] = df['ZipCode'].astype('str')
df['ZipCode'] = [None if len(code)<5 else code for code in df['ZipCode']]
df = df.sort_values(by=['Major', 'ZipCode'])
df['ZipCode'] = df['ZipCode'].fillna(method='ffill')
df['ZipCode'] = [code[0:5] for code in df['ZipCode']]

In [ ]:
# Check x if have the relevant data to answer question
# [x]Higher square footage increases home sale price1, 2
# [x]Having a porch increases home sale price3, 4
# [x]Having a beachfront or lakefront increases home sale price5
# [x]The house filling a higher proportion of the overall lot decreases home sale price6
# [x]The cost per square foot is lower in duplexes than in single-family homes7
# [x]The presence of a nuisance (power lines, traffic noise, airport noise) decreases home sale price1, 5

## Graphical EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

### Plot some distributions and box plots and Remove Outliers

In [ ]:
sns.scatterplot(df['SqFtLot'], df['SalePrice'])

In [ ]:
sns.distplot(df['SalePrice'])

In [ ]:
sns.boxplot(df['SalePrice'])

In [ ]:
df2 = df[df['SalePrice'].between(df['SalePrice'].quantile(.001), df['SalePrice'].quantile(.996))]

In [ ]:
sns.distplot(df2['SalePrice'])

In [ ]:
sns.boxplot(df2['SalePrice'])

In [ ]:
sns.distplot(df2['SqFtLot'])

In [ ]:
sns.boxplot(df['SqFtLot'])

In [ ]:
df2 = df2[df2['SqFtLot'].between(df2['SqFtLot'].quantile(.001), df2['SqFtLot'].quantile(.999))]

In [ ]:
sns.distplot(df2['SqFtLot'])

In [ ]:
sns.boxplot(df2['SqFtLot'])

In [ ]:
sns.scatterplot(df2['SqFtLot'], df2['SalePrice'])

In [ ]:
df2=df2.reset_index(drop=True)

In [ ]:
sns.barplot(df2.Is_WaterFrontLocation, df2.SalePrice);

In [ ]:
chart = sns.barplot(df2.WaterFrontLocation, df2.SalePrice)
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right');

In [ ]:
sns.barplot(df2['Has_Nuisance'], df2['SalePrice']);

In [ ]:
sns.barplot(df2['Nuisances'], df2['SalePrice']);

In [ ]:
sns.barplot(df2['Has_Porch'], df2['SalePrice']);

In [ ]:
df2['SalePrice'].max()

In [ ]:
len(df2)

In [ ]:
df2.to_csv('../../data/clean/KingCountyHousingData.csv')